# MUSIC (MUltiple SIgnal Classification) Notebook

The MUSIC (MUltiple SIgnal Classification) algorithm is a rather generic and well-know algorithm in signal process used for the detection of multiple waves within an array of sensors [Schmidt, 1986].  MUSIC was originally used in SuperDARN by Samson et al. [1990] and Bristow et al. [1994] for the estimation of the characteristics of Medium Scale Traveling Ionospheric Disturbances (MSTIDs) detected by SuperDARN Radars.  These papers have inspired this implementation of the MUSIC algorithm.

While this code has been written with SuperDARN detected MSTIDs in mind, it may be useful for working with other wave-like perturbations moving across the field of view of some geophysical instrument.

This notebook demonstrates the use of the DaViTPy MUSIC module for MSTID parameter estimation.

The DaViTPy MUSIC module is used by creating a musicArray object, which includes built-in methods for keeping track of all steps in the processing algorithm.  As the data is processed, all preceeding variants of the data are saved within the object and a history is created.  This makes it very easy to see what has been done with the data at any step along the way.

*Written by N.A. Frissell, 4 November 2013.*

**References**

Bristow, W. A., R. A. Greenwald, and J. C. Samson (1994), Identification of high-latitude acoustic gravity wave sources using the Goose Bay HF Radar, J. Geophys. Res., 99(A1), 319–331, doi:10.1029/93JA01470.

Samson, J. C., R. A. Greenwald, J. M. Ruohoniemi, A. Frey, and K. B. Baker (1990), Goose Bay radar observations of Earth-reflected, atmospheric gravity waves in the high-latitude ionosphere, J. Geophys. Res., 95(A6), 7693–7709, doi:10.1029/JA095iA06p07693.

Schmidt, R.O., "Multiple emitter location and signal parameter estimation," Antennas and Propagation, IEEE Transactions on, vol.34, no.3, pp.276,280, Mar 1986, doi:10.1109/TAP.1986.1143830.

In [7]:
#Import the modules we need.
%matplotlib inline
import datetime

from matplotlib import pyplot as plt 
import numpy as np
from music import musicArray,musicDataObj
from plotting.musicPlot import musicRTI
from radDataRead import radDataOpen
from music import defineLimits, filterTimes,beamInterpolation,timeInterpolation,determineRelativePosition
from plotting.musicPlot import musicFan,timeSeriesMultiPlot,plotRelativeRanges

import pydarn
import pydarnio

# MUSIC Processing Example

## Loading Data and Basic Plotting

In [9]:
#Choose the radar and time of the data that we want to look at.
#Then, connect to the data source using the standard DaViTPy
#pydarn.sdio.radDataOpen() routine.
# channel = 'a'
bmnum = 7
# cp = None
fileType = 'fitacf'
# filtered = False
# src = None
rad='bks'
syear = 2010
smonth = 11
sday = 19
shour = 14
sminute = 11

eyear = 2010
emonth = 11
eday = 19
ehour = 16
eminute = 11

# local_dirfmt = '/home/fran/code/SuperdarnW3usr/ForGitRepo/data_vt_bks/sd-data/2010/fitacf/bks/'
# local_dirfmt = f'/home/fran/code/SuperdarnW3usr/ForGitRepo/{syear}/{fileType}/data_vt_{rad}/'
# fileName = f'/home/fran/code/SuperdarnW3usr/ForGitRepo/{syear}/{fileType}/data_vt_{rad}/20101119.1401.00.bks.fitacf.bz2'

local_dirfmt = f'/sd-data/{syear}/{fileType}/{rad}/'

sTime = datetime.datetime(syear,smonth,sday,shour,sminute)
eTime = datetime.datetime(eyear,emonth,eday,ehour,eminute)

myPtr = radDataOpen(sTime=sTime,radcode=rad, eTime=eTime, channel=None, bmnum=None, cp=None,
                fileType=fileType, filtered=False, src=None, fileName=None,
                noCache=False, local_dirfmt=local_dirfmt, local_fnamefmt=None,
                local_dict=None, remote_dirfmt=None, remote_fnamefmt=None,
                remote_dict=None, remote_site=None, username=None,
                password=None, port=None, tmpdir=None, remove=False,
                try_file_types=True)

ERROR:root:Config entry DAVIT_LOCAL_FNAMEFMT not set,  using default: ['{date}.{hour}......{radar}.{ftype}', '{date}.{hour}......{radar}.{channel}.{ftype}']
NoneType: None


RunTime: 3.249680995941162
RunTime: 2.972712516784668
Runtime: 6.351561784744263


cat: /tmp/sd/20101119.1401.00.bks.fitacf/tmp/sd/20101119.1601.00.bks.fitacf: Not a directory


In [ ]:
# print(myPtr)

In [3]:
# Now create the data object.

# By creating this object, data is taken from the SuperDARN database and rearranged into a numpy.array with
# the shape (Nr Times, Nr Beams, Nr Gates).  Missing data is stored as np.nan.

# By default, this command only loads in data where the ground scatter flag is True.

# The fovModel='GS' indicates to calculate the field-of-view coordinates in using the ground-scatter mapping
# formula.  This is standard when looking at MSTIDs using ground scatter.  See Bristow et al. [1994] for details.
# dataObj_IS     = musicArray(myPtr,fovModel='IS')
dataObj     = musicArray(myPtr,fovModel='IS')

/home/w2naf/code/MUSIC/radar/radFov.py: 790: SyntaxWarning: "is" with a literal. Did you mean "=="?
/home/w2naf/code/MUSIC/radar/radFov.py: 790: SyntaxWarning: "is" with a literal. Did you mean "=="?
/home/w2naf/code/MUSIC/radar/radFov.py: 790: SyntaxWarning: "is" with a literal. Did you mean "=="?
/home/w2naf/code/MUSIC/radar/radFov.py: 790: SyntaxWarning: "is" with a literal. Did you mean "=="?


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# We can list all of the data sets stored in this object.  Right now there is only one data set, but
# as we go along, each step of processing will create a new data set.

# Each data set is simply stored as an attribute of the dataObj, but the names of all of the data sets can be
# accessed through the get_data_sets() method.
dataObj.get_data_sets()

In [ ]:
# Information about the data set is saved in it's metadata dictionary.  Some of the information in the metadata
# dictionary is used to control certain plotting and processing variables in this module.

dataObj.DS000_originalFit.printMetadata()

In [ ]:
# The music module also keeps track of each step of processing in using the data set's history attribute.

dataObj.DS000_originalFit.printHistory()

In [ ]:
# The 'active' attribute of dataObj is a reference to the most recently used data set of dataObj.  It is also
# the default data set used by all DaViTPy MUSIC processing and plotting routines.  You can see this command
# produces the same result as the cell above.

dataObj.active.printHistory()

In [ ]:
# We can create an RTI plot and a fan plot from the data we loaded.
# You can see that both range gate and geographic latitude are given on the y-axis, and the solar terminator
# is also shaded in.  The terminator does not go below approximately range gate 10.  This is because the ground
# scatter mapping formula is not defined for close-in range gates.

fig = musicRTI(dataObj)

In [ ]:
# We can also make a fan plot.

plotTime = datetime.datetime(2010,11,19,14,40)
fig = musicFan(dataObj,time=plotTime)


## Data Selection

In [ ]:
# We want to focus on just the data that contains the MSTID we are looking for, so we can apply some limits.

# proc.music.defineLimits() does not actually modify the data, it just marks the data by putting an an entry
# into the dataObj.active.metadata dictionary.  The limits will be applied later.  Right now, you can see
# what data will be eliminated by making a new RTI plot.

defineLimits(dataObj,gateLimits=[10,30])
fig = musicRTI(dataObj)

In [ ]:
# We also to restrict the amount of time we process.  Before actually running the music algorithm, we
# will be filtering the data using a FIR filter that will eat up data at the beginning and end of the filter.
# We can calculate exactly how much time that will be if we know some of the filter characteristics and
# the time resolution of the data.  This can then be used to give us new start and end times.

# For now, let's say we are going to use a filter with 101 taps and a dataset with 120 s resolution.
numtaps = 101
timeres = 120

#Let's also say that we are interested in the MSTID feature between 1400 and 1600 UT.
sTime_of_interest = datetime.datetime(2010,11,19,14,11)
eTime_of_interest = datetime.datetime(2010,11,19,16,11)

#Now calculate the new start and end times...
new_times = filterTimes(sTime_of_interest, eTime_of_interest, timeres, numtaps)
defineLimits(dataObj,timeLimits=new_times)

fig = musicRTI(dataObj)

In [ ]:
# Now we apply the limits and replot once more.
# Note that many of the processing routines will automatically call applyLimits() before they
# run the processing algorithm.
# prob
dataObj.active.applyLimits()
fig = musicRTI(dataObj)

In [ ]:
# Note that a new data set was created when we applied the limits.
dataObj.get_data_sets()

In [ ]:
# Also note that the history of the new object was updated.
dataObj.active.printHistory()

## Data Processing

### Interpolation and Cartesian Coordinates

In [ ]:
# Before we feed into the MUSIC Algorithm, we don't want our data to have any gaps in time or space.
# Let's start by interpolating in space along the beams.
beamInterpolation(dataObj)

In [ ]:
import cartopy.crs as ccrs
proj= ccrs.PlateCarree()
fig = plt.figure(figsize=(20,10))
ax  = fig.add_subplot(121, projection=proj,aspect='auto')
musicFan(dataObj,plotZeros=True,dataSet='originalFit',axis=ax,time=plotTime)
ax  = fig.add_subplot(122, projection=proj, aspect='auto')
musicFan(dataObj,plotZeros=True,axis=ax,time=plotTime)

In [ ]:
%debug
# We also want to interpolate in time.  timeres=120 [seconds] was set in an earlier cell.
timeInterpolation(dataObj,timeRes=timeres)

In [ ]:
timeSeriesMultiPlot(dataObj,dataSet='timeInterpolated',dataSet2='beamInterpolated')

In [ ]:
# We also want need to calculate a local cartesian grid for each cell we are going to use.
determineRelativePosition(dataObj)

In [ ]:
#The black cell marks the center of the array.
fig = plotRelativeRanges(dataObj,time=plotTime)

### Filtering

In [ ]:
# Now filter the data.
# numtaps=101 was set in an above cell.  The cutoff_frequencies are in Hz.
from music import filter
filt = filter(dataObj, numtaps=numtaps, cutoff_low=0.0003, cutoff_high=0.0012)

In [ ]:
# At this point, the data has been filtered and saved to a new data set in dataObj.
# Before we look at the data, let's look at the transfer function and impulse response of the data.

fig = filt.plotTransferFunction(xmax=0.004)
fig = filt.plotImpulseResponse()

In [ ]:
# Let's look at the filtered RTI plot.  We should set autoScale to True since the magnitudes will be much
# lower than the original data.

fig = musicRTI(dataObj,autoScale=True)

In [ ]:
# You can see that the filter already marked off the data that you shouldn't use.
# Just applyLimits() and replot.
dataObj.active.applyLimits()
fig = musicRTI(dataObj,autoScale=True)

In [ ]:
# Look at a fan plot.
fig = musicFan(dataObj,time=plotTime,autoScale=True)

### Spectral Analysis

In [ ]:
# We have now run all of the processing needed to feed the data into the spectral analysis and MUSIC.
# Let's print the history just to recap what we have done.
dataObj.active.printHistory()

In [ ]:
# First thing to do is to calculate the FFT of every cell...
from music import calculateFFT
calculateFFT(dataObj)

In [ ]:
# We can look at the spectrum of select cells...
from plotting.musicPlot import spectrumMultiPlot
spectrumMultiPlot(dataObj,xlim=(-0.0025,0.0025))
spectrumMultiPlot(dataObj,plotType='magnitude',xlim=(0,0.0025))
spectrumMultiPlot(dataObj,plotType='phase',xlim=(0,0.0025))

In [ ]:
# We can also plot the full spectrum.  Here, every FFT bin contains 16 slices,
# showing the data for each of the 16 radar beams from left to right.
# Range gates are shown on the y-axis.
from plotting.musicPlot import plotFullSpectrum
plotFullSpectrum(dataObj,xlim=(0,0.00175))

### MUSIC Processing

In [ ]:
# Calculate the cross-spectral matrix Dlm.
from music import calculateDlm
calculateDlm(dataObj)

In [ ]:
# We can look at the Dlm matrix.  This usually isn't necessary for routine processing, but
# it is good to know where the numbers are going...
from plotting.musicPlot import plotDlm
plotDlm(dataObj)

In [ ]:
# Now, we finally run detect the horizontal wave numbers.
from music import calculateKarr
calculateKarr(dataObj)

In [ ]:
# We can then plot the final result.
from plotting.musicPlot import plotKarr
plotKarr(dataObj)

In [ ]:
# Again, we can see all of the history gone into processing this plot.
dataObj.active.printHistory()

### Feature/Signal Detection

In [ ]:
# We can use the image processing library in scikit-image to automatically find the peaks representing signals.
from music import detectSignals
detectSignals(dataObj)
plotKarr(dataObj)

In [ ]:
#All of the parameters connected with these signals is located in the sigDectect attribute.
dataObj.active.sigDetect.info

In [ ]:
# It is possible to manually add a signal peak in.
# All appropriate associated wave parameters will be calculated.
from music import add_signal
add_signal(0.013,-0.015,dataObj)
plotKarr(dataObj)

In [ ]:
dataObj.active.sigDetect.info

In [ ]:
#Signals can also be removed from the database.
from music import del_signal
del_signal(4,dataObj)
plotKarr(dataObj)

In [ ]:
# To get some insight on how the signal processing algorithm works, we can plot the detected groups.
# You can adjust the threshold and neighborhood keywords on pydarn.proc.music.detectSignals() to tweak
# the autodetection.
from plotting.musicPlot import plotKarrDetected
plotKarrDetected(dataObj)